<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/b4_public_datasets_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using public datasets with TF Datasets

In [1]:
!pip install tensorflow-addons

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

from tensorflow.keras import layers

tfds.__version__

'4.0.1'

In [3]:
mnist_data = tfds.load('fashion_mnist')
type(mnist_data), mnist_data

(dict,
 {'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
  'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>})

In [4]:
for item in mnist_data:
  print(type(item), item)

<class 'str'> test
<class 'str'> train


If you want to load these splits into a dataset containing the actual data, you can simply specify the split you want in the tfds.load command, like this:

In [5]:
mnist_train = tfds.load(name='fashion_mnist', split='train')
assert isinstance(mnist_train, tf.data.Dataset)
type(mnist_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In this instance, we we a `PrefetchDataset` object, which we can iterate through to inspect the data. One nice feature is that we can apply `take(1)` and get the first record.

In [6]:
item = next(iter(mnist_train.take(1)))
print(type(item))
print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


In [7]:
image = item['image']
print(type(image))
print(image.shape)
print(image[0:0])

<class 'tensorflow.python.framework.ops.EagerTensor'>
(28, 28, 1)
tf.Tensor([], shape=(0, 28, 1), dtype=uint8)


In [8]:
label = item['label']
print(type(label))
print(label)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(2, shape=(), dtype=int64)


In [9]:
mnist_test, info = tfds.load(name='fashion_mnist', with_info='true')
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=3.0.1,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

## Using TFDS with Keras Model

In [10]:
mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(type(train_images))

<class 'numpy.ndarray'>


When using TFDS the code is very similar, but with some minor changes. The Keras datasets gave us `ndarray` that worked natively in `model.fit`. However, with TFDS we will need to do a little conversion work.

In [11]:
(train_images, train_labels), (test_images, test_labels) = \
  tfds.as_numpy(
      tfds.load('fashion_mnist',
                split=['train', 'test'],
                batch_size=-1,
                as_supervised=True))
print(type(train_images))

<class 'numpy.ndarray'>


In [12]:
# we need to rescale our images before feeding them into the network
# train_images = train_images * 1.0/255.0
# test_images = test_images * 1.0/255.0
# skipping this rescaling step in favor of adding rescaling directly
# into the model pipeline(see layers...Rescaling) 

model = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1.0/255.0),
  layers.Flatten(input_shape=(28, 28, 1)),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.fit(
    train_images,
    train_labels,
    epochs=5
)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5283 - accuracy: 0.8138
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3998 - accuracy: 0.8541
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3670 - accuracy: 0.8657
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3464 - accuracy: 0.8729
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3294 - accuracy: 0.8794


The data is batched and shuffled to make training more effective?

## Human-or-Horses Model

In [13]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)
 
train_batches = data.shuffle(100).batch(10)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_batches, 
    epochs=10,
    validation_data=validation_batches
)

Epoch 1/10
103/103 [==============================] - 5s 49ms/step - loss: 0.3218 - accuracy: 0.8627 - val_loss: 1.8059 - val_accuracy: 0.8477
Epoch 2/10
103/103 [==============================] - 5s 45ms/step - loss: 0.1534 - accuracy: 0.9484 - val_loss: 1.9836 - val_accuracy: 0.7773
Epoch 3/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0628 - accuracy: 0.9747 - val_loss: 3.3394 - val_accuracy: 0.7969
Epoch 4/10
103/103 [==============================] - 5s 44ms/step - loss: 0.0671 - accuracy: 0.9757 - val_loss: 0.4579 - val_accuracy: 0.8984
Epoch 5/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0354 - accuracy: 0.9912 - val_loss: 2.4246 - val_accuracy: 0.8086
Epoch 6/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0074 - accuracy: 0.9971 - val_loss: 4.1770 - val_accuracy: 0.7734
Epoch 7/10
103/103 [==============================] - 5s 45ms/step - loss: 0.0973 - accuracy: 0.9708 - val_loss: 1.6175 - val_accuracy: 0.8750

## Using Mapping Functions for Augmentation

In [14]:
def augment_images(image, label):
  image = tf.image.random_flip_left_right(image)
  return image, label

In [15]:
train = data.map(augment_images)

In [16]:
train_batches = train.shuffle(100).batch(32)
model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_batches, 
    epochs=10,
)

Epoch 1/10
33/33 [==============================] - 4s 125ms/step - loss: 0.3559 - accuracy: 0.8267
Epoch 2/10
33/33 [==============================] - 4s 119ms/step - loss: 0.1247 - accuracy: 0.9484
Epoch 3/10
33/33 [==============================] - 4s 121ms/step - loss: 0.0529 - accuracy: 0.9805
Epoch 4/10
33/33 [==============================] - 4s 120ms/step - loss: 0.0259 - accuracy: 0.9922
Epoch 5/10
33/33 [==============================] - 4s 120ms/step - loss: 0.0260 - accuracy: 0.9922
Epoch 6/10
33/33 [==============================] - 4s 121ms/step - loss: 0.0229 - accuracy: 0.9961
Epoch 7/10
33/33 [==============================] - 4s 120ms/step - loss: 0.0340 - accuracy: 0.9854
Epoch 8/10
33/33 [==============================] - 4s 119ms/step - loss: 0.0619 - accuracy: 0.9805
Epoch 9/10
33/33 [==============================] - 4s 120ms/step - loss: 0.0033 - accuracy: 0.9981
Epoch 10/10
33/33 [==============================] - 4s 118ms/step - loss: 0.0021 - accuracy: 1.0000

## Sofisticated Augmentation

In [17]:
def resize(image, label):
  image = tf.image.resize(image, [300, 300]) 
  return image, label 

In [20]:
def augment_image(image, label):
  image, label = resize(image, label)
  # random crop back to the original size
  image = tf.image.random_crop(image, size=[300, 300, 3])
  # random brightness
  image = tf.image.random_brightness(image, max_delta=0.5)
  # image = tfa.image.rotate(image, 40, interpolation='NEAREST')
  image = tf.clip_by_value(image, 0, 1)
  return image, label

In [21]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = (
    train
    .shuffle(1000)
    .map(augment_image, num_parallel_calls=AUTOTUNE)
    .batch(32)
    .prefetch(AUTOTUNE)
)

In [22]:
model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_ds, 
    epochs=10
)

Epoch 1/10
 2/33 [>.............................] - ETA: 2s - loss: 0.6928 - accuracy: 0.4844WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0298s vs `on_train_batch_end` time: 0.0859s). Check your callbacks.


33/33 [==============================] - 2s 61ms/step - loss: 0.6942 - accuracy: 0.4956
Epoch 2/10
33/33 [==============================] - 2s 60ms/step - loss: 0.6932 - accuracy: 0.5131
Epoch 3/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6931 - accuracy: 0.5131
Epoch 4/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6933 - accuracy: 0.5131
Epoch 5/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6929 - accuracy: 0.5131
Epoch 6/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6929 - accuracy: 0.5131
Epoch 7/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6928 - accuracy: 0.5131
Epoch 8/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6929 - accuracy: 0.5131
Epoch 9/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6929 - accuracy: 0.5131
Epoch 10/10
33/33 [==============================] - 2s 59ms/step - loss: 0.6929 - accuracy: 0.5131
